In [1]:
f = open('bạn bè.txt','r')
sentences = f.readlines()

In [2]:
# !pip install vncorenlp

In [3]:
!mkdir -p vncorenlp/models/wordsegmenter

In [4]:
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar

In [5]:
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab

In [6]:
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr

In [7]:
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 

mv: cannot stat 'VnCoreNLP-1.1.1.jar': No such file or directory


In [8]:
!mv vi-vocab vncorenlp/models/wordsegmenter/

mv: cannot stat 'vi-vocab': No such file or directory


In [9]:
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

mv: cannot stat 'wordsegmenter.rdr': No such file or directory


In [10]:
!pip install vncorenlp

In [12]:
import os
os.system("java -version")

32512

In [11]:
import re
# from vncorenlp import VnCoreNLP
import vncorenlp

rdrsegmenter = vncorenlp.VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 



questions = []
answers = []
for sent in sentences:
    q_a = sent.split('__eou__')
    for i in range(len(q_a)):
        q_a[i] = re.sub('^ | $','',q_a[i]) #bỏ 2 dấu cách đầu và cuối câu
    try:
        questions.append(q_a[0])
        if q_a[1] == '':
            answers.append('None')
        else:
            answers.append(q_a[1])
    except Exception:
        continue

print(len(questions))
print(len(answers))
print(questions[0:5])
print(answers[0:5])

FileNotFoundError: Java was not found, please install JRE or JDK >= 1.8 first.

In [ ]:
from vncorenlp import VnCoreNLP

#Word2Vec

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
tokenizer = nltk.RegexpTokenizer(r"\w+")

questions_tokenize = [rdrsegmenter.tokenize(sent.lower())[-1] for sent in questions]
answers_tokenize = [rdrsegmenter.tokenize(sent.lower())[-1] for sent in answers]

In [ ]:
# for sent in range(len(answers)):
#   try:
#     # print(rdrsegmenter.tokenize(sent.lower())[-1])
#     rdrsegmenter.tokenize(answers[sent].lower())[-1]
#     continue
#   except:
#     print(sent)

In [ ]:
questions_tokenize[:5]

In [ ]:
answers_tokenize[:5]

In [ ]:
print(len(questions_tokenize))
print(len(answers_tokenize))

In [ ]:
# train_text_tokenized = [nltk.word_tokenize(sent) for sent in X_train+y_train]
# test_text_tokenized = [nltk.word_tokenize(sent) for sent in X_test+y_test]

In [ ]:
from gensim.models import Word2Vec
modelw2v = Word2Vec(sentences=questions_tokenize+answers_tokenize, size=100, window=5, sg=1, min_count=1)

In [ ]:
print("Vocab size: " + str(len(modelw2v.wv.vocab)))

In [ ]:
modelw2v.wv.most_similar('bạn')

In [ ]:
def w2v(array_tokenize):
  temp = {}
  for i,sent in enumerate(array_tokenize):
    sent_tokenize = []
    for word in sent:
      try:
        sent_tokenize.append(modelw2v.wv[word])
      except Exception:
        continue
    try:
      temp[i] = sum(sent_tokenize)/len(sent_tokenize)
    except Exception:
      error_idx.add(i)
      continue
  return temp

In [ ]:
import numpy as np

error_idx = set()


temp_X = w2v(questions_tokenize)

temp_Y = w2v(answers_tokenize)



X = []

for e in temp_X:
  if e in error_idx:
    continue
  X.append(temp_X[e])

Y = []

for e in temp_Y:
  if e in error_idx:
    continue
  Y.append(temp_Y[e])


print(np.array(X).shape)
print(np.array(Y).shape)
print(X[0:2])
print(Y[0:2])

In [ ]:
string = 'Nhậu ko cu'

In [ ]:
string = rdrsegmenter.tokenize(string.lower())[-1]
print(string)

In [ ]:
sent_tokenize = []
for word in string:
  try:
    sent_tokenize.append(modelw2v.wv[word])
  except Exception:
    continue

input = sum(sent_tokenize)/len(sent_tokenize)

In [ ]:
import math as m
def euclid_distance(X, Y):
  total = 0
  for i in range(len(X)):
    total += m.pow(X[i]-Y[i],2)
  return m.sqrt(total)

def get_best_index(input, X):
  min = -(m.inf)
  idx = 0
  for i,q in enumerate(X):
    if euclid_distance(input, q) < min:
      min = euclid_distance(input, q)
      idx = i
  return i

In [ ]:
print(answers[get_best_index(input, X)])

In [ ]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# from keras.utils import to_categorical

# y_train = to_categorical(train_labels)
# y_test = to_categorical(test_labels)